# Decorator

* 透過引數來接受其他函式的可呼叫函式
* Decorator 可以讓被修飾函式執行一些工作並回傳
* 將被修飾函式轉換成其他函式可呼叫物件

In [4]:
def deco(func):
    def inner():
        print('runung inner()')
    return inner
@deco
def target():
    print('running target()')


In [5]:
target()

runung inner()


In [6]:
target

<function __main__.deco.<locals>.inner()>

# Python 何時執行 decorator

In [7]:
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()  # <9>

running register(<function f1 at 0x7f7ff0221158>)
running register(<function f2 at 0x7f7ff02212f0>)
running main()
registry -> [<function f1 at 0x7f7ff0221158>, <function f2 at 0x7f7ff02212f0>]
running f1()
running f2()
running f3()


# 用 decorator 改寫 strategy

In [9]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# BEGIN STRATEGY_BEST4

promos = []  # <1>

def promotion(promo_func):  # <2>
    promos.append(promo_func)
    print(promos.append)
    return promo_func

@promotion  # <3>
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):  # <4>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

<built-in method append of list object at 0x7f7ff0246688>
<built-in method append of list object at 0x7f7ff0246688>
<built-in method append of list object at 0x7f7ff0246688>


In [60]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

# 變數範圍規則

說明 closure 前導

In [67]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3
6


In [66]:
b = 6
def f2(a):
    print(a)
    print(b)
    b =9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [26]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
9


In [27]:
b

9

In [23]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [24]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [25]:
dis(f3)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# Closure

* 其中L是local，E是enclosing，G是global，B是build-in
* 通常python搜尋變數的順序是L->E->G->B

In [32]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [61]:
avg = Averager()
avg(10)

10.0

In [62]:
avg(11)

10.5

In [63]:
avg.__closure__

AttributeError: 'Averager' object has no attribute '__closure__'

In [28]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [29]:
avg = make_averager()
avg(10)

10.0

In [30]:
avg(11)

10.5

In [31]:
avg(12)

11.0

In [33]:
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [36]:
avg.__closure__

(<cell at 0x7f7ff0300288: list object at 0x7f7ff028a8c8>,)

In [37]:
avg.__closure__[0].cell_contents

[10, 11, 12]

# nonlocal 宣告(Python 3)
python 處理方式把 count total 存成 list dict 並把物件指派為 free value

In [38]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [77]:
# 數字, 字串 tuple 只能讀取無法更新
avg = make_averager()
avg(10)

10.0

In [41]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

In [42]:
avg = make_averager()
avg(10)

10.0

# 實作簡單修飾器

In [43]:
import time

def clock(func):
    # 定義內部函式讓他可以接受引數
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        # 回傳內部函式，來替換被修飾函式
        return result
    return clocked

In [44]:
import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12314892s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000048s] factorial(1) -> 1
[0.00002074s] factorial(2) -> 2
[0.00003672s] factorial(3) -> 6
[0.00005221s] factorial(4) -> 24
[0.00008392s] factorial(5) -> 120
[0.00011349s] factorial(6) -> 720
6! = 720


In [45]:
import clockdeco_demo
clockdeco_demo.factorial.__name__

'clocked'

In [46]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pair = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

# 標準程式庫的修飾器

In [48]:
from clockdeco import clock
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
if __name__=='__main__':
    print(fibonacci(6))

[0.00000048s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00038028s] fibonacci(2) -> 1
[0.00000024s] fibonacci(1) -> 1
[0.00000024s] fibonacci(0) -> 0
[0.00000024s] fibonacci(1) -> 1
[0.00002980s] fibonacci(2) -> 1
[0.00005889s] fibonacci(3) -> 2
[0.00047016s] fibonacci(4) -> 3
[0.00000024s] fibonacci(1) -> 1
[0.00000024s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00002861s] fibonacci(2) -> 1
[0.00005698s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000024s] fibonacci(1) -> 1
[0.00002861s] fibonacci(2) -> 1
[0.00000024s] fibonacci(1) -> 1
[0.00000024s] fibonacci(0) -> 0
[0.00000024s] fibonacci(1) -> 1
[0.00002909s] fibonacci(2) -> 1
[0.00005746s] fibonacci(3) -> 2
[0.00027871s] fibonacci(4) -> 3
[0.00036764s] fibonacci(5) -> 5
[0.00086713s] fibonacci(6) -> 8
8


In [49]:
import functools

from clockdeco import clock

@functools.lru_cache() # <1>
@clock  # <2>
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))


[0.00000048s] fibonacci(0) -> 0
[0.00001001s] fibonacci(1) -> 1
[0.00051713s] fibonacci(2) -> 1
[0.00000095s] fibonacci(3) -> 2
[0.00058246s] fibonacci(4) -> 3
[0.00000072s] fibonacci(5) -> 5
[0.00063014s] fibonacci(6) -> 8
8


# 單次指派通用函式

In [50]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # <2>
def _(text):            # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

# 修飾器堆疊

```python
@d1
@d2
def f():
    print('f')
def f():
    print('f')
f = d1(d2(f))
```

# 具有參數的修飾器

In [51]:
registry = []
def register(func):
    print('runing register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('runing main()')
print('registry ->', registry)
f1()

runing register(<function f1 at 0x7f7ff0303d90>)
runing main()
registry -> [<function f1 at 0x7f7ff0303d90>]
running f1()


# 具有參數的註冊修飾器

In [68]:
registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7f7fe16c0bf8>)
running register(active=True)->decorate(<function f2 at 0x7f7fe16c08c8>)


In [76]:
import registration_param
registration_param.registry

{<function registration_param.f2()>}

<function registration_param.register(active=True)>

# 具有參數的 clock 修飾器

In [54]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # <1>
    def decorate(func):      # <2>
        def clocked(*_args): # <3>
            t0 = time.time()
            _result = func(*_args)  # <4>
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # <6>
            print(fmt.format(**locals()))  # <7>
            return _result  # <8>
        return clocked  # <9>
    return decorate  # <10>

if __name__ == '__main__':

    @clock()  # <11>
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12315512s] snooze(0.123) -> None
[0.12309790s] snooze(0.123) -> None
[0.12314320s] snooze(0.123) -> None


In [55]:
# clockdeco_param_demo1.py
import time
from clockdeco_param import clock

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12314915657043457s
snooze: 0.12308979034423828s
snooze: 0.12314939498901367s


In [58]:
# clockdeco_param_demo2.py
import time
from clockdeco_param import clock

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)


snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
